In [1]:
import os
import json
import logging
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import xarray as xr

from scipy.spatial import KDTree
import folium

In [2]:
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

# BTS coords data

In [3]:
importcols = ['cell', 'lat', 'lon', 'Height', 'Azimuth']
bts = pd.read_csv(os.path.join(config['data_path'], 'spatial_vars_old.csv'), usecols=importcols, index_col='cell')
print(bts.shape)
bts.head()

(484, 4)


,lat,lon,Height,Azimuth
cell,,,,
322_0,51.93255,6.57633,28.3,80
322_1,51.93255,6.57633,28.3,190
322_2,51.93255,6.57633,28.3,320
168_0,52.63140,4.72566,30.5,60
168_1,52.63140,4.72566,30.5,180


# Import meteo data (API)

In [4]:
with open("../config.json") as config_file:
    config = json.load(config_file)
    api_key = config["meteo_api"]

api_url = "https://api.dataplatform.knmi.nl/open-data/v1/datasets/Actuele10mindataKNMIstations/versions/2/files"
headers = {"Authorization": f"Bearer {api_key}"}

In [5]:
# Define date range
start_date = "2023-09-03T00:00:00+00:00"
end_date = "2023-09-03T23:30:00+00:00"
params = {"begin": start_date, "end": end_date, "orderBy": "created"}

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# Ensure the target directory exists
save_dir = config['data_path']+"/meteo_raw"
os.makedirs(save_dir, exist_ok=True)

In [7]:
# Fetch files in the date range
response = requests.get(api_url, headers=headers, params=params)
response.raise_for_status()
files = response.json().get("files", [])

if not files:
    logger.info("No files found in the specified date range.")
    exit()

# Download up to 10 files in the date range
max_files_to_download = 5
downloaded_files_count = 0

for file_info in files:
    if downloaded_files_count >= max_files_to_download:
        break
    filename = file_info["filename"]
    logger.info(f"Downloading: {filename}")

    # Get the download URL
    file_url = f"{api_url}/{filename}/url"
    response = requests.get(file_url, headers=headers)
    response.raise_for_status()
    download_url = response.json().get("temporaryDownloadUrl")

    # Download and save file
    local_path = os.path.join(save_dir, filename)
    file_data = requests.get(download_url)
    file_data.raise_for_status()

    with open(local_path, "wb") as f:
        f.write(file_data.content)

    logger.info(f"Saved: {local_path}")

    downloaded_files_count += 1

INFO:__main__:Downloading: KMDS__OPER_P___10M_OBS_L2_202309030000.nc
INFO:__main__:Saved: /Users/jakobtjurlik/Library/CloudStorage/OneDrive-Personal/Desktop/Study/MSc_Tilburg/MSc Thesis/code/radio-signal-anomalies/data/meteo_raw/KMDS__OPER_P___10M_OBS_L2_202309030000.nc
INFO:__main__:Downloading: KMDS__OPER_P___10M_OBS_L2_202309030010.nc
INFO:__main__:Saved: /Users/jakobtjurlik/Library/CloudStorage/OneDrive-Personal/Desktop/Study/MSc_Tilburg/MSc Thesis/code/radio-signal-anomalies/data/meteo_raw/KMDS__OPER_P___10M_OBS_L2_202309030010.nc
INFO:__main__:Downloading: KMDS__OPER_P___10M_OBS_L2_202309030020.nc
INFO:__main__:Saved: /Users/jakobtjurlik/Library/CloudStorage/OneDrive-Personal/Desktop/Study/MSc_Tilburg/MSc Thesis/code/radio-signal-anomalies/data/meteo_raw/KMDS__OPER_P___10M_OBS_L2_202309030020.nc
INFO:__main__:Downloading: KMDS__OPER_P___10M_OBS_L2_202309030030.nc
INFO:__main__:Saved: /Users/jakobtjurlik/Library/CloudStorage/OneDrive-Personal/Desktop/Study/MSc_Tilburg/MSc Thesis/c

In [8]:
# List files in the directory
nc_files = [f for f in os.listdir(save_dir) if f.endswith(".nc")]
print(f"Found {len(nc_files)} NetCDF files.")

# Open the first available file
if nc_files:
    file_name = nc_files[0]  # Extract the filename
    file_path = os.path.join(save_dir, file_name)
    print(f"Opening file: {file_name}")

    # Open NetCDF file with xarray
    dataset = xr.open_dataset(file_path)
    print(dataset)
else:
    print("No NetCDF files found.")

Found 7 NetCDF files.
Opening file: KMDS__OPER_P___10M_OBS_L2_202309030010.nc
<xarray.Dataset> Size: 65kB
Dimensions:      (station: 69, time: 1)
Coordinates:
  * station      (station) <U5 1kB '06201' '06203' '06204' ... '78873' '78990'
  * time         (time) datetime64[ns] 8B 2023-09-03T00:10:00
Data variables: (12/103)
    stationname  (station) <U36 10kB ...
    lat          (station) float64 552B ...
    lon          (station) float64 552B ...
    height       (station) float64 552B ...
    D1H          (station, time) float64 552B ...
    dd           (station, time) float64 552B ...
    ...           ...
    zm           (station, time) float64 552B ...
    iso_dataset  |S1 1B ...
    product      |S1 1B ...
    projection   |S1 1B ...
    nhc          |S1 1B ...
    za           |S1 1B ...
Attributes:
    featureType:  timeSeries
    Conventions:  CF-1.4
    title:        KMDS__OPER_P___10M_OBS_L2
    institution:  Royal Netherlands Meteorological Institute (KNMI)
    source: 

# Matching meteo stations to BTS

In [9]:
# Convert relevant weather station metadata to DataFrame
weather_stations = pd.DataFrame({
    "station_id": dataset["station"].values,   # Station IDs
    "lat": dataset["lat"].values,              # Latitude
    "lon": dataset["lon"].values,              # Longitude
    "height": dataset["height"].values         # Height above sea level
})

# Display the first few rows
print("Weather Station Data (head):")
print(weather_stations.head())

Weather Station Data (head):
  station_id      lat     lon  height
0      06201  54.3256  2.9358   42.70
1      06203  52.3600  3.3417   41.84
2      06204  53.2694  3.6278   41.80
3      06205  55.3992  3.8103   48.35
4      06207  53.6144  4.9603   45.31


In [10]:
# Build KDTree for weather stations
weather_tree = KDTree(weather_stations[["lat", "lon"]].values)

# Find the nearest weather station for each BTS cell
distances, nearest_idx = weather_tree.query(bts[["lat", "lon"]].values)

# Add matched weather station info to BTS DataFrame
bts["nearest_station"] = weather_stations.iloc[nearest_idx]["station_id"].values
bts["station_lat"] = weather_stations.iloc[nearest_idx]["lat"].values
bts["station_lon"] = weather_stations.iloc[nearest_idx]["lon"].values
bts["station_height"] = weather_stations.iloc[nearest_idx]["height"].values
bts["distance_km"] = distances * 111  # Convert degrees to ~km

# Display results
print("BTS Cell Data with Matched Weather Station:")
print(bts.head())

BTS Cell Data with Matched Weather Station:
            lat      lon  Height  Azimuth nearest_station  station_lat  \
cell                                                                     
322_0  51.93255  6.57633    28.3       80           06283      52.0678   
322_1  51.93255  6.57633    28.3      190           06283      52.0678   
322_2  51.93255  6.57633    28.3      320           06283      52.0678   
168_0  52.63140  4.72566    30.5       60           06233      52.4819   
168_1  52.63140  4.72566    30.5      180           06233      52.4819   

       station_lon  station_height  distance_km  
cell                                             
322_0       6.6567           29.07    17.463337  
322_1       6.6567           29.07    17.463337  
322_2       6.6567           29.07    17.463337  
168_0       4.7258           -1.60    16.594507  
168_1       4.7258           -1.60    16.594507  


## Check result

In [ ]:
# Define the center of the map (Netherlands approx center)
map_center = [52.2, 5.5]
m = folium.Map(location=map_center, zoom_start=8)

# Add BTS Cells to the map (Blue)
for _, row in bts.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
        popup=f"BTS Cell {row.name}",
    ).add_to(m)

# Add Weather Stations to the map (Red)
for _, row in weather_stations.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=f"Weather Station {row['station_id']}",
    ).add_to(m)

# Save as an interactive HTML file
output_path = os.path.join(config["data_path"], "bts_weather_map.html")
m.save(output_path)

In [20]:
# Print the total number of unique BTS cells
total_unique_bts_cells = bts.index.nunique()
print("Total BTS Cells:", total_unique_bts_cells)

# Print the total number of unique weather station IDs
total_unique_station_ids = weather_stations["station_id"].nunique()
print("Total Weather Station IDs:", total_unique_station_ids)

# Check if all weather stations are matched
unique_matched_stations = bts["nearest_station"].nunique()
all_stations_matched = unique_matched_stations == total_unique_station_ids
print("Total Matched Weather Stations:", unique_matched_stations)

Total BTS Cells: 484
Total Weather Station IDs: 69
Total Matched Weather Stations: 37


## Export

In [16]:
output_csv_path = os.path.join(config['data_path'], 'spatial_new.csv')
bts.to_csv(output_csv_path)